In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery-data-through-gemini/data_set.csv


In [2]:
# load text 

filename = '/kaggle/input/llm-prompt-recovery-data-through-gemini/data_set.csv'
file = open(filename, 'rt')
text = file.read()
file.close()

In [3]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

2024-04-07 07:57:55.733939: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 07:57:55.734058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 07:57:55.872820: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
# Define documents
# Assign the 'Prompt' column to the docs variable

docs = data['Prompt'].tolist()

In [29]:
# Integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[20, 30, 15, 43, 21, 25, 33, 23, 23, 20, 3, 23, 46, 31, 15, 3, 29, 14, 4, 25, 23, 34, 7, 20, 30, 15, 43, 6, 29, 14, 25, 23, 41, 35, 31, 11, 7, 11, 31, 22, 46, 46, 17, 38, 26, 46, 17, 23, 46, 15, 20, 30, 15, 43, 46, 29, 22, 20, 17, 43, 30, 15, 1, 47, 30, 20, 3, 46, 29, 22, 30, 4, 35, 32, 44, 6, 38, 28, 40, 21, 13, 13], [20, 30, 15, 43, 21, 25, 33, 23, 23, 20, 3, 23, 46, 31, 15, 3, 29, 14, 4, 25, 23, 34, 7, 20, 30, 15, 43, 6, 29, 14, 25, 23, 41, 35, 31, 11, 7, 11, 31, 22, 46, 46, 17, 38, 26, 46, 17, 23, 46, 15, 20, 30, 15, 43, 46, 29, 22, 20, 17, 43, 30, 15, 1, 47, 30, 20, 3, 46, 29, 22, 30, 4, 35, 32, 44, 6, 38, 28, 40, 21, 13, 13], [20, 30, 15, 43, 21, 25, 33, 23, 23, 20, 3, 23, 46, 31, 15, 3, 29, 14, 4, 25, 23, 34, 7, 20, 30, 15, 43, 6, 29, 14, 25, 23, 41, 35, 31, 11, 7, 11, 31, 22, 46, 46, 17, 38, 26, 46, 17, 23, 46, 15, 20, 30, 15, 43, 46, 29, 22, 20, 17, 43, 30, 15, 1, 47, 30, 20, 3, 46, 29, 22, 30, 4, 35, 32, 44, 6, 38, 28, 40, 21, 13, 13], [20, 30, 15, 43, 21, 25, 33, 23, 23, 20

In [30]:
# Pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[40 21 13 13]
 [40 21 13 13]
 [40 21 13 13]
 ...
 [28 33 14 41]
 [28 33 14 41]
 [28 33 14 41]]


In [33]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the CSV file
csv_file = '/kaggle/input/llm-prompt-recovery-data-through-gemini/data_set.csv'
data = pd.read_csv(csv_file)

# Extract text data from the 'Question' column
train_docs = data['Prompt'].tolist()

# Handle NaN values by replacing them with an empty string
train_docs = [str(doc) for doc in train_docs]

# Create a tokenizer object
tokenizer = Tokenizer()
# Fit the tokenizer on the training documents
tokenizer.fit_on_texts(train_docs)
# Encode the training documents to sequences of integers
encoded_docs = tokenizer.texts_to_sequences(train_docs)

# Pad sequences to ensure uniform length
max_length = max([len(s.split()) for s in train_docs])  # Find the maximum length
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')  # Pad sequences

# Define training labels
# Assuming 'Question' column contains the labels for each document
ytrain = data['Question'].values

In [34]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Read the CSV file
data = pd.read_csv('/kaggle/input/llm-prompt-recovery-data-through-gemini/data_set.csv')

# Split the data into features (X) and target (y)
X = data['Prompt']
y = data['Question']  # Assuming 'Question' column contains the target labels

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad the tokenized sequences
max_length = 100  # Define the maximum length of sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8))  # Remove input_length parameter
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train_padded, y_train, epochs=10, verbose=2)

# Evaluate
loss, acc = model.evaluate(X_test_padded, y_test, verbose=0)
print('Test Accuracy: %.2f%%' % (acc * 100))

Epoch 1/10
33/33 - 1s - 27ms/step - accuracy: 9.6339e-04 - loss: -5.0120e+02
Epoch 2/10
33/33 - 0s - 2ms/step - accuracy: 9.6339e-04 - loss: -5.0247e+03
Epoch 3/10
33/33 - 0s - 2ms/step - accuracy: 9.6339e-04 - loss: -2.1682e+04
Epoch 4/10
33/33 - 0s - 2ms/step - accuracy: 9.6339e-04 - loss: -5.6708e+04
Epoch 5/10
33/33 - 0s - 4ms/step - accuracy: 9.6339e-04 - loss: -1.1426e+05
Epoch 6/10
33/33 - 0s - 2ms/step - accuracy: 9.6339e-04 - loss: -1.9253e+05
Epoch 7/10
33/33 - 0s - 4ms/step - accuracy: 9.6339e-04 - loss: -2.9605e+05
Epoch 8/10
33/33 - 0s - 2ms/step - accuracy: 9.6339e-04 - loss: -4.2000e+05
Epoch 9/10
33/33 - 0s - 4ms/step - accuracy: 9.6339e-04 - loss: -5.6468e+05
Epoch 10/10
33/33 - 0s - 4ms/step - accuracy: 9.6339e-04 - loss: -7.3164e+05
Test Accuracy: 0.00%


The dataset of prompt is very low result accuracy .There are no classified correct by the model.